# 14. Analisis Varians Multivariat (MANOVA)

## Pengenalan MANOVA (Multivariate Analysis of Variance)

MANOVA adalah perluasan dari ANOVA yang memungkinkan analisis simultan dari beberapa variabel dependen. Berbeda dengan ANOVA yang hanya menganalisis satu variabel dependen, MANOVA dapat menganalisis hubungan antara variabel independen dan beberapa variabel dependen secara bersamaan.

### Konsep Dasar MANOVA

**MANOVA (Multivariate Analysis of Variance)** adalah teknik statistik yang digunakan untuk:
- Menguji perbedaan mean dari beberapa variabel dependen antar kelompok
- Menganalisis efek dari satu atau lebih variabel independen pada beberapa variabel dependen
- Mengontrol Type I error rate ketika melakukan multiple comparisons

### Asumsi MANOVA

1. **Multivariate Normality**: Data mengikuti distribusi normal multivariat
2. **Homogeneity of Variance-Covariance Matrices**: Matriks varians-kovarians sama antar kelompok
3. **Independence**: Observasi saling bebas
4. **Linearity**: Hubungan linear antar variabel dependen
5. **Absence of Outliers**: Tidak ada outlier yang ekstrem

### Jenis-jenis MANOVA

1. **One-way MANOVA**: Satu variabel independen dengan beberapa variabel dependen
2. **Two-way MANOVA**: Dua variabel independen dengan beberapa variabel dependen
3. **Factorial MANOVA**: Lebih dari dua variabel independen

### Statistik Uji MANOVA

- **Wilks' Lambda (Λ)**: Paling umum digunakan
- **Pillai's Trace**: Lebih robust terhadap pelanggaran asumsi
- **Hotelling's Trace**: Untuk sampel kecil
- **Roy's Largest Root**: Untuk kasus khusus

### Interpretasi Hasil

- **F-test**: Menguji signifikansi secara keseluruhan
- **Effect Size**: Mengukur kekuatan efek
- **Post-hoc Analysis**: Analisis lanjutan untuk menentukan kelompok mana yang berbeda


In [ ]:
# Import libraries yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import multivariate_normal
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries berhasil diimport!")


## 1. One-Way MANOVA

### Contoh: Analisis Prestasi Akademik

Kita akan menganalisis perbedaan prestasi akademik (matematika, fisika, kimia) berdasarkan metode pembelajaran yang berbeda.


In [ ]:
# Generate data simulasi untuk One-Way MANOVA
np.random.seed(42)

# Parameter untuk setiap kelompok
n_per_group = 30
groups = ['Traditional', 'Interactive', 'Online']
n_groups = len(groups)

# Mean dan covariance untuk setiap kelompok
means = {
    'Traditional': [75, 70, 68],
    'Interactive': [85, 80, 82],
    'Online': [78, 75, 77]
}

covariance = np.array([
    [25, 15, 12],
    [15, 20, 10],
    [12, 10, 18]
])

# Generate data
data = []
for group in groups:
    group_data = np.random.multivariate_normal(means[group], covariance, n_per_group)
    group_df = pd.DataFrame(group_data, columns=['Math', 'Physics', 'Chemistry'])
    group_df['Method'] = group
    data.append(group_df)

# Combine data
df_manova = pd.concat(data, ignore_index=True)

print("Data One-Way MANOVA:")
print(df_manova.head())
print(f"\nShape: {df_manova.shape}")
print(f"\nGroup sizes: {df_manova['Method'].value_counts()}")


In [ ]:
# Visualisasi data One-Way MANOVA
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Box plot untuk setiap variabel dependen
for i, subject in enumerate(['Math', 'Physics', 'Chemistry']):
    ax = axes[i//2, i%2]
    sns.boxplot(data=df_manova, x='Method', y=subject, ax=ax)
    ax.set_title(f'{subject} Scores by Teaching Method')
    ax.set_xlabel('Teaching Method')
    ax.set_ylabel(f'{subject} Score')

# Scatter plot matrix
ax = axes[1, 1]
scatter_data = df_manova[['Math', 'Physics', 'Chemistry']].values
colors = ['red', 'green', 'blue']
for i, group in enumerate(groups):
    group_data = df_manova[df_manova['Method'] == group][['Math', 'Physics', 'Chemistry']].values
    ax.scatter(group_data[:, 0], group_data[:, 1], c=colors[i], label=group, alpha=0.7)
ax.set_xlabel('Math Score')
ax.set_ylabel('Physics Score')
ax.set_title('Math vs Physics Scores')
ax.legend()

plt.tight_layout()
plt.show()

# Descriptive statistics
print("\nDescriptive Statistics by Group:")
desc_stats = df_manova.groupby('Method')[['Math', 'Physics', 'Chemistry']].agg(['mean', 'std'])
print(desc_stats)


### Implementasi One-Way MANOVA Manual

Kita akan mengimplementasikan MANOVA secara manual untuk memahami konsep matematisnya.


In [ ]:
def one_way_manova(data, group_col, dependent_vars):
    """
    Implementasi manual One-Way MANOVA
    
    Parameters:
    data: DataFrame dengan data
    group_col: nama kolom variabel independen
    dependent_vars: list nama kolom variabel dependen
    
    Returns:
    dict dengan hasil MANOVA
    """
    # Extract data
    Y = data[dependent_vars].values
    groups = data[group_col].unique()
    n = len(data)
    p = len(dependent_vars)  # number of dependent variables
    k = len(groups)  # number of groups
    
    # Calculate overall mean
    grand_mean = np.mean(Y, axis=0)
    
    # Calculate group means
    group_means = {}
    group_sizes = {}
    for group in groups:
        group_data = data[data[group_col] == group][dependent_vars].values
        group_means[group] = np.mean(group_data, axis=0)
        group_sizes[group] = len(group_data)
    
    # Calculate Within-Groups Sum of Squares and Cross-Products (W)
    W = np.zeros((p, p))
    for group in groups:
        group_data = data[data[group_col] == group][dependent_vars].values
        for obs in group_data:
            diff = obs - group_means[group]
            W += np.outer(diff, diff)
    
    # Calculate Between-Groups Sum of Squares and Cross-Products (B)
    B = np.zeros((p, p))
    for group in groups:
        diff = group_means[group] - grand_mean
        B += group_sizes[group] * np.outer(diff, diff)
    
    # Calculate Total Sum of Squares and Cross-Products (T)
    T = W + B
    
    # Calculate Wilks' Lambda
    wilks_lambda = np.linalg.det(W) / np.linalg.det(T)
    
    # Calculate F-statistic
    s = min(p, k-1)
    m = (n - k - p - 1) / 2
    t = (p**2 + (k-1)**2 - 5) / 2
    
    if p**2 + (k-1)**2 - 5 != 0:
        f_stat = ((1 - wilks_lambda**(1/s)) / (wilks_lambda**(1/s))) * ((s*m - t + 1) / (s*(k-1)))
    else:
        f_stat = ((1 - wilks_lambda) / wilks_lambda) * ((n - k - p + 1) / (k - 1))
    
    # Degrees of freedom
    df1 = s * (k - 1)
    df2 = s * (n - k - p + 1)
    
    # p-value
    p_value = 1 - stats.f.cdf(f_stat, df1, df2)
    
    return {
        'wilks_lambda': wilks_lambda,
        'f_statistic': f_stat,
        'df1': df1,
        'df2': df2,
        'p_value': p_value,
        'within_sscp': W,
        'between_sscp': B,
        'total_sscp': T
    }

# Jalankan One-Way MANOVA
manova_result = one_way_manova(df_manova, 'Method', ['Math', 'Physics', 'Chemistry'])

print("=== ONE-WAY MANOVA RESULTS ===")
print(f"Wilks' Lambda: {manova_result['wilks_lambda']:.4f}")
print(f"F-statistic: {manova_result['f_statistic']:.4f}")
print(f"Degrees of freedom: ({manova_result['df1']}, {manova_result['df2']})")
print(f"p-value: {manova_result['p_value']:.6f}")
print(f"Significant: {'Yes' if manova_result['p_value'] < 0.05 else 'No'}")
